# Курс «Введение в нейронные сети»

## Урок 7. Детектирование объектов

### Домашняя работа к уроку 7

#### Задание 1

Сделайте краткий обзор какой-нибудь научной работы посвященной тому или иному алгоритму для object detection, который не рассматривался на уроке. Проведите анализ: Чем отличается выбранная вами на рассмотрение архитектура нейронной сети от других архитектур? В чем плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при применении данной архитектуры на практике?

Рассматриваем статью "Review: RetinaNet — Focal Loss (Object Detection)"
https://towardsdatascience.com/review-retinanet-focal-loss-object-detection-38fba6afabe4

**Обзор: RetinaNet - потеря фокуса (обнаружение объектов)**

В работе